### **Dataset Cleaning - Emily's Sets**

In this notebook, I (Emily) will explain my data cleaning process for the datasets I was assigned to work on.  My collaborators and I worked independently on this portion of the work and used the methods that worked the best for us personally.  However, we all discussed and agreed upon which data to delete, preserve, or modify.

One decision that became clear as we cleaned the data was that we unfortunately needed to exclude Alpine County from the dataset.  Alpine County is very small, and therefore, almost all of its social condition data were suppressed for privacy.  After that decision was made, Alpine County was removed from any further datasets we cleaned.  It remained, however, in datasets that had already been cleaned, and was removed for good at a later step.

In [69]:
# Imports
import numpy as np
import os
import pandas as pd

# Thanks to stackoverflow for this tip to suppress SettingWithCopyWarning
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
pd.options.mode.chained_assignment = None 

In [70]:
os.chdir('./../02_data_eks')
os.listdir()

['.Rhistory',
 '00_ignore',
 '01_original_datasets',
 '02_cleaned_datasets',
 '03_output',
 '04_data-dictionaries',
 'ca_dropout_and_predictors_v6_eks.csv']

#### Abortion Costs

In [71]:
# Read it in
abortion_costs = pd.read_csv('./01_original_datasets/abortions_funded_costs.csv')

# Rename columns
col_names = [col.lower().replace(' ', '_') for col in abortion_costs.columns]
abortion_costs.columns = col_names

# Get the shape
abortion_costs.shape

(815, 5)

In [72]:
# 2015 only
abortion_costs = abortion_costs[abortion_costs['calendar_year']==2015]

# Real counties only
abortion_costs = abortion_costs[abortion_costs['county']!='Unknown']
abortion_costs = abortion_costs[abortion_costs['county']!='Total'] 

# Get the shape
abortion_costs.shape

(114, 5)

In [73]:
# Take a look at the column counts - this will help me identify unnecessary features

# Define a function to use this again later
def col_counts(df):
  '''A function to print the number of unique values for every column in a dataframe
  Arg: 
    {df}, a dataframe
  Return: 
    Nothing, the purpose of this function is to see it in the output.
  Raise:
    Hopefully not!'''
  for i in list(df.columns):
    print('='*20)
    print(i)
    print(df[i].nunique())

# Run it
col_counts(abortion_costs)

calendar_year
1
delivery_system
2
county
58
total_expenditures
55
date_of_data
1


In [74]:
# Look for missing data
abortion_costs.isna().sum()

calendar_year          0
delivery_system        0
county                 0
total_expenditures    58
date_of_data           0
dtype: int64

Just the one column has NAs, and it is exactly the number of counties.

In [75]:
# Take a look at the column counts if I were to drop those
col_counts(abortion_costs.dropna())

calendar_year
1
delivery_system
1
county
56
total_expenditures
55
date_of_data
1


It looks like an entire category of `delivery_system` is missing.

In [76]:
# Check that hypothesis
abortion_costs['total_expenditures'].isna().groupby(abortion_costs['delivery_system']).sum()

delivery_system
Fee-for-Service     0
Managed Care       58
Name: total_expenditures, dtype: int64

In [77]:
# Check that hypothesis
abortion_costs['total_expenditures'].isna().groupby(abortion_costs['delivery_system']).value_counts(dropna = False)

delivery_system  total_expenditures
Fee-for-Service  False                 56
Managed Care     True                  58
Name: count, dtype: int64

Two counties seem to be missing from the Fee-for-Service level that are present in the Managed Care level, but they're NA there.  They're not suppressed for small numbers or anything; they just aren't present in the dataset at all.

In [78]:
# Drop the unnecessary category (rows)
abortion_costs.dropna(inplace = True)

# Drop unnecessary columns
drop_cols = ['calendar_year', 'delivery_system', 'date_of_data']
abortion_costs.drop(columns = drop_cols, inplace = True)

# Get the shape
abortion_costs.shape

(56, 2)

In [79]:
# Save to csv 
abortion_costs.to_csv('./03_output/abortion_costs_eks.csv', index = False)

The datasets that resulted from my initial cleaning, that went on to be part of the final analysis, can be found in `./02_cleaned_datasets`.  I have edited this code to send them to `./03_output` instead, to prevent these files from being overwritten over and over.  This is the same throughout this notebook.  Curious readers - or at least those using Git Bash on a PC - can run the following command through their terminal to confirm that the files are the same.

```diff 02_data/03_output/<dataset name>_eks.csv 02_data/02_cleaned_datasets/<dataset name>_simplified_eks.csv```

#### Abortion Counts

In [80]:
# Read it in
abortion_counts = pd.read_csv('./01_original_datasets/abortions_funded_counts.csv')

# Rename columns
col_names = [col.lower().replace(' ', '_') for col in abortion_counts.columns]
col_names = [col.lower().replace('-', '_') for col in col_names]
abortion_counts.columns = col_names
abortion_counts.columns

# Get the shape
abortion_counts.shape

(1231, 7)

In [81]:
# 2015 only
abortion_counts = abortion_counts[abortion_counts['calendar_year']==2015]

# Real counties only
abortion_counts = abortion_counts[abortion_counts['county']!='Unknown']
abortion_counts = abortion_counts[abortion_counts['county']!='Statewide'] 

# Get the shape
abortion_counts.shape

(174, 7)

In [82]:
# Take a look at the column counts
col_counts(abortion_counts)

calendar_year
1
county
58
delivery_system
3
total_abortion_related_services
140
annotation_code
2
annotation_description
2
date_of_data
1


In [83]:
# Look for missing data
abortion_counts.isna().sum()

calendar_year                        0
county                               0
delivery_system                      0
total_abortion_related_services     25
annotation_code                    149
annotation_description             149
date_of_data                         0
dtype: int64

This dataset did not come with a data dictionary, but the annotation columns indicate that the numbers from some counties were suppressed for privacy reasons (i.e., they were so small).  I hypothesized that that was why some columns had missing values.

In [84]:
# Check those rows out
abortion_counts[abortion_counts['total_abortion_related_services'].isna()].head(2)

,calendar_year,county,delivery_system,total_abortion_related_services,annotation_code,annotation_description,date_of_data
178,2015,Alpine,Managed Care,NaN,1.0,Cell suppressed for small numbers,1/12/2022
179,2015,Alpine,Total,NaN,1.0,Cell suppressed for small numbers,1/12/2022


In [85]:
print(f'''The total number of rows in this dataset is {abortion_counts.shape[0]}.
Are the annotation columns only filled in when the count was suppressed and NA elsewhere, 
and the count columns only filled in when they weren't suppresed, and NA elsewhere?
{abortion_counts.shape[0]==(
    abortion_counts['total_abortion_related_services'].isna().sum() + abortion_counts[
    'annotation_code'].isna().sum())}''')

The total number of rows in this dataset is 174.
Are the annotation columns only filled in when the count was suppressed and NA elsewhere, 
and the count columns only filled in when they weren't suppresed, and NA elsewhere?
True


In [86]:
# What is in these annotation columns?
abortion_counts['annotation_code'].value_counts(dropna = False)

annotation_code
NaN    149
1.0     16
2.0      9
Name: count, dtype: int64

In [87]:
abortion_counts['annotation_description'].value_counts(dropna = False)

annotation_description
NaN                                       149
Cell suppressed for small numbers          16
Cell suppressed for complementary cell      9
Name: count, dtype: int64

The missing values correspond to suppressions.  I left them in at this stage, and my collaborators and I later decided to impute them with 0s.  Although it is unlikely that the real number was 0 (in fact, 0s were reported in this data), it is a reasonable value to use given that the cause of the missingness is that the true values were so small.

The annotation columns, as received, list two types of suppression.  For our purposes, I combined these two types at this stage.  In a later stage, in the interest of reducing our dimensions, we dropped the suppression markers entirely.  Because the values were imputed with 0s, rather than an estimate within range of the unsuppressed columns, these rows were essentially already marked as different. 

In [88]:
abortion_counts.columns

Index(['calendar_year', 'county', 'delivery_system',
       'total_abortion_related_services', 'annotation_code',
       'annotation_description', 'date_of_data'],
      dtype='object')

In [89]:
# Recode these annotation columns
abortion_counts.drop(columns = ['annotation_description'], inplace = True)
abortion_counts['annotation_code'] = abortion_counts['annotation_code'].fillna(0)
annot_2s = abortion_counts['annotation_code']==2
abortion_counts.loc[annot_2s, 'annotation_code'] = 1
abortion_counts.rename(columns = {
    'annotation_code': 'abortion_rs_count_total_suppressed',
    'total_abortion_related_services': 'abortion_rs_count_total'}, inplace = True)

Data were listed on separate rows depending on the type of payment Medi-Cal made for the abortion related services, but in the interest of minimizing our dimensions, we choose to use the total count only.

In [90]:
# Drop unnecessary rows
abortion_counts = abortion_counts[abortion_counts['delivery_system']=='Total']
abortion_counts = abortion_counts[abortion_counts['county']!='Alpine']

# Drop unnecessary columns
abortion_counts.drop(
    columns = ['calendar_year', 'delivery_system', 'date_of_data'], inplace = True)

# Get the shape
abortion_counts.shape

(57, 3)

In [91]:
# Save to csv
abortion_counts.to_csv('./03_output/abortion_counts_eks.csv', index = False)

# EMILY DELETE THIS YOU-SPECIFIC LINE
```diff 02_data_eks/03_output/<dataset name>_eks.csv 02_data_eks/02_cleaned_datasets/<dataset name>_simplified_eks.csv```

#### Daycare Slots

In [92]:
# Read it in
daycare_slots = pd.read_csv('./01_original_datasets/daycare_slots.csv')

# Rename columns
col_names = {
    'reportyear': 'report_year', 'geotypevalue': 'geotype_value'}
daycare_slots.rename(columns = col_names, inplace = True)

# Get the shape
daycare_slots.shape

(20101, 28)

In [93]:
# Drop redundant columns
drop_cols = ['ind_id', 'ind_definition', 'race_eth_code',
       'race_eth_name', 'version', 'region_name',
       'strata_name_code', 'strata_name', 'ca_decile']
daycare_slots.drop(columns = drop_cols, inplace = True)

# Get the shape
daycare_slots.shape

(20101, 19)

In [95]:
# Real counties only
daycare_slots = daycare_slots[daycare_slots['geotype']=='CO']

# Get the shape
daycare_slots.shape

(116, 19)

In [98]:
# Take a look at the column counts
col_counts(daycare_slots)

report_year
1
geotype
1
geotype_value
58
geoname
58
county_fips
58
county_name
58
region_code
14
strata_level_name_code
2
strata_level_name
2
facility_capacity
108
total_pop
116
rate_slots
111
ll_95ci
111
ul_95ci
111
se
111
rse
108
ca_rr
111
no_facility
72
pct_nonwhite
115


In [99]:
# Look for missing data
daycare_slots.isna().sum()

report_year               0
geotype                   0
geotype_value             0
geoname                   0
county_fips               0
county_name               0
region_code               0
strata_level_name_code    0
strata_level_name         0
facility_capacity         5
total_pop                 0
rate_slots                5
ll_95ci                   5
ul_95ci                   5
se                        5
rse                       5
ca_rr                     5
no_facility               5
pct_nonwhite              0
dtype: int64

This dataset contains two rows per county - one for infant (0-2) daycare availability, and one for child (2-5) daycare availability.  To condense these into one row, I first broke the dataframe into two dataframes (one per type), gave the columns within those dataframes different names, then merged them back together.

In [101]:
# Divide the df
df1 = daycare_slots[daycare_slots['strata_level_name_code']==1] # child
df2 = daycare_slots[daycare_slots['strata_level_name_code']==2] # infant

In [102]:
# Take a look at the column counts
col_counts(df1)

report_year
1
geotype
1
geotype_value
58
geoname
58
county_fips
58
county_name
58
region_code
14
strata_level_name_code
1
strata_level_name
1
facility_capacity
58
total_pop
58
rate_slots
58
ll_95ci
58
ul_95ci
58
se
58
rse
58
ca_rr
58
no_facility
50
pct_nonwhite
58


In [103]:
col_counts(df2)

report_year
1
geotype
1
geotype_value
58
geoname
58
county_fips
58
county_name
58
region_code
14
strata_level_name_code
1
strata_level_name
1
facility_capacity
51
total_pop
58
rate_slots
53
ll_95ci
53
ul_95ci
53
se
53
rse
51
ca_rr
53
no_facility
33
pct_nonwhite
58


In [104]:
# Drop unnecessary columns
drop_cols = ['county_fips', 'region_code',
    'geoname', 'report_year', 'strata_level_name_code', 
    'strata_level_name', 'geotype', 'geotype_value',
    'll_95ci', 'ul_95ci', 'se', 'rse', 'ca_rr']

df1.drop(columns = drop_cols, inplace = True) #child
df2.drop(columns = drop_cols, inplace = True) #infant

# Get the shapes
print(df1.shape)
print(df2.shape)

(58, 6)
(58, 6)


In [108]:
# Rename the columns
infant_cols = ['county_name', 'infant_facility_capacity',
    'infant_total_pop', 'infant_rate_slots',
    'infant_num_facility', 'infant_pct_nonwhite']

child_cols = ['county_name', 'child_facility_capacity', 
    'child_total_pop', 'child_rate_slots', 
    'child_num_facility', 'child_pct_nonwhite']

df1.columns = child_cols
df2.columns = infant_cols

# Get the shapes
print(df1.shape)
print(df2.shape)

(58, 6)
(58, 6)


In [29]:
# Save to csv 

#### E-cigarettes

In [30]:
# Read it in


# Rename columns


# Get the shape


In [31]:
# 2015 only


# Real counties only


# Get the shape


In [32]:
# Take a look at the column counts

In [33]:
# Look for missing data

In [34]:
# Drop unnecessary rows


# Drop unnecessary columns


# Get the shape


In [35]:
# Save to csv 

#### Poverty Rate

In [36]:
# Read it in


# Rename columns


# Get the shape


In [37]:
# 2015 only


# Real counties only


# Get the shape


In [38]:
# Take a look at the column counts

In [39]:
# Look for missing data

In [40]:
# Drop unnecessary rows


# Drop unnecessary columns


# Get the shape


In [41]:
# Save to csv 

#### Suicide Rate

In [42]:
# Read it in


# Rename columns


# Get the shape


In [43]:
# 2015 only


# Real counties only


# Get the shape


In [44]:
# Take a look at the column counts

In [45]:
# Look for missing data

In [46]:
# Drop unnecessary rows


# Drop unnecessary columns


# Get the shape


In [47]:
# Save to csv 

#### Graduation Cohort & Dropout Rate

In [48]:
# Read it in


# Rename columns


# Get the shape


In [49]:
# 2015 only


# Real counties only


# Get the shape


In [50]:
# Take a look at the column counts

In [51]:
# Look for missing data

In [52]:
# Drop unnecessary rows


# Drop unnecessary columns


# Get the shape


In [53]:
# Save to csv 

#### Unemployment Rate

In [54]:
# Read it in


# Rename columns


# Get the shape


In [55]:
# 2015 only


# Real counties only


# Get the shape


In [56]:
# Take a look at the column counts

In [57]:
# Look for missing data

In [58]:
# Drop unnecessary rows


# Drop unnecessary columns


# Get the shape


In [59]:
# Save to csv 

#### Additional Dimensionality Reduction